# Control of inverted pendulum with Reinforce algorithm


### Policy Gradient

In this code, I will implement vanilla policy gradient algorithm (REINFORCE) covered in the lecture. You will work on i) a function approximator, ii) computing action, iii) collecting samples, iV) training the agent, V) plotting the resutls. 


***Complete the missing operations and test your implemented algorithm on the Gym environment.***

***Software requirements:***
- Python >= 3.6
- Tensorflow version <= 1.15.3 (1.X version)
- OpenAI Gym

- Training the agent (policy) can take long time. It is recomended to start solving the problems earlier.

- Save any plots you generated in this notebook. The grade will be given based on the plots you showed.



Make sure the packages you installed meet the requirements.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal


In [2]:
import gym
gym.__version__

'0.13.0'

## 1.1 Tensorflow Implementation

We will be implementing policy gradient algorithm using Tensorflow 1.X., which simply updates the parameters of policy from obtaining gradient estimates. The core of policy gradient is to design a function approximator, computing actions, collecting samples, and training the policy. In the below cell, you are encouraged to fill in the components that are missing. ***Your tasks*** are 

1. Complete the 'create_model' method to output the mean value for diagonal Guassian policy. Covariance is already defined in the model, so focus on creating neural network model.

2. Complete the 'action_op' method to calculate and return the actions for diagonal Gaussian policy. The applied action should be $\pi(s) = \pi_{\text{mean}}(s) + exp(logstd) * \mathcal{N}(0,1)$

***Hints***:
- Some useful tensorflow classes and methods include: 'tf.exp', 'tf.random_normal'



In [3]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
import numpy as np

class PolicyOpt(object):
    def __init__(self, env, linear=False, stochastic=True, hidden_size=32, nonlinearity=None):
        """
        Minimal PyTorch port preserving the original algorithmic structure:
        - Gaussian policy pi(a|s) with mean = network(s), diagonal std as a trainable parameter
        - REINFORCE-compatible API surface (compute_action, log_likelihoods_eval, compute_expected_return)
        """
        self.env = env
        self.stochastic = stochastic
        self.obs_dim = int(np.prod(env.observation_space.shape))
        self.act_dim = int(np.prod(env.action_space.shape))
        self.act_low  = torch.as_tensor(env.action_space.low,  dtype=torch.float32)
        self.act_high = torch.as_tensor(env.action_space.high, dtype=torch.float32)

        # Build policy network
        layers = []
        last = self.obs_dim
        if linear:
            layers.append(nn.Linear(last, self.act_dim))
        else:
            layers += [nn.Linear(last, hidden_size), nn.Tanh(), nn.Linear(hidden_size, self.act_dim)]
        self.policy_net = nn.Sequential(*layers)

        # Log-std as a free parameter (state-independent)
        self.log_std = nn.Parameter(torch.full((self.act_dim,), -0.5))

        # Optimizer placeholder (used in REINFORCE.train)
        self.optimizer = None

        # Default discount
        self.gamma = 0.99

    def _dist(self, obs_tensor: torch.Tensor) -> Normal:
        mu = self.policy_net(obs_tensor)
        std = torch.exp(self.log_std).expand_as(mu)
        return Normal(mu, std)

    def compute_action(self, s):
        """Return an action (np.ndarray) given a single state or batch of states."""
        s_t = torch.as_tensor(np.array(s, dtype=np.float32))
        if s_t.ndim == 1:
            s_t = s_t.unsqueeze(0)
        dist = self._dist(s_t)
        if self.stochastic:
            a = dist.sample()
        else:
            a = dist.mean
        a = a.clamp(self.act_low, self.act_high)
        return a.detach().cpu().numpy()

    def log_likelihoods_eval(self, states, actions):
        """Return log pi(a|s) for each (s,a) pair as a 1D numpy array."""
        s_t = torch.as_tensor(np.array(states, dtype=np.float32))
        a_t = torch.as_tensor(np.array(actions, dtype=np.float32))
        if s_t.ndim == 1: s_t = s_t.unsqueeze(0)
        if a_t.ndim == 1: a_t = a_t.unsqueeze(0)
        dist = self._dist(s_t)
        logp = dist.log_prob(a_t)  # (batch, act_dim)
        logp = logp.sum(dim=-1)    # sum over action dims
        return logp.detach().cpu().numpy()

    def compute_expected_return(self, samples, normalize=False):
        """
        samples: list of dicts with key 'reward' -> list/array of rewards for each episode
        returns: flattened reward-to-go across episodes (concatenated), optionally normalized
        """
        all_G = []
        for ep in samples:
            r = np.asarray(ep["reward"], dtype=np.float32)
            G = np.zeros_like(r, dtype=np.float32)
            g = 0.0
            for t in range(len(r)-1, -1, -1):
                g = r[t] + float(self.gamma) * g
                G[t] = g
            all_G.append(G)
        out = np.concatenate(all_G, axis=0)
        if normalize:
            out = (out - out.mean()) / (out.std() + 1e-8)
        return out


## 1.2 Tensorflow Interpretation

In order to test your implementation of the **stochastic policy**, run the below cell. The task is to interpret the code you implemented in previous section. If you implement correctly, you can see the value_1 and value_2.

***Question: How do you interpret value_1 and value_2 below cell?***


In [4]:
TEST_ENV = gym.make("Pendulum-v0")
alg = PolicyOpt(TEST_ENV, linear=False)

import numpy as np, torch
input_1 = np.array([[0, 1, 2]], dtype=np.float32)

# Put input on the same device as the model
model_device = next(alg.policy_net.parameters()).device
with torch.no_grad():
    value_1 = alg.policy_net(
        torch.as_tensor(input_1, dtype=torch.float32, device=model_device)
    ).detach().cpu().numpy()

value_2 = alg.compute_action(input_1)

print("policy mean (value_1):", value_1)
print("sampled/greedy action (value_2):", value_2)


policy mean (value_1): [[-0.3831538]]
sampled/greedy action (value_2): [[0.29644418]]


In [5]:
value_1


array([[-0.3831538]], dtype=float32)

Answer:

In [6]:
value_2

array([[0.29644418]], dtype=float32)

Answer:

## 1.3 Implement Policy Gradient

In this section, we will implement REINFORCE algorithm presented in the lecture. As a review, the objective is optimize the parameters $\theta$ of some policy $\pi_\theta$ so that the expected return

\begin{equation}
J(\theta) = \mathbb{E} \bigg\{ \sum_{t=0}^T \gamma^t r(s_{t},a_{t}) \bigg\}
\end{equation}

is optimized. In this algorithm, this is done by calculating the gradient $\nabla_\theta J$ and applying a gradient descent method to find a better policy.

\begin{equation}
\theta ' = \theta + \alpha \nabla_\theta J(\theta)
\end{equation}

In the lecture, we derive how we compute $\nabla_{\theta} J(\theta)$. We can rewrite our policy gradient as:

\begin{equation}
\nabla_\theta J (\theta) \approx \frac{1}{N} \sum_{i=0}^{N} \bigg( \sum_{t=0}^{T} \nabla_\theta \log \pi_\theta (a_{it} | s_{it}) \bigg) \bigg( \sum_{t=0}^T \gamma^{t}r_i(t) \bigg)
\end{equation}

Finally, taking into account the causality principle discussed in class, we are able to simplifiy the gradient estimate such as:

\begin{equation}
\nabla_\theta J (\theta) \approx \frac{1}{N} \sum_{i=0}^{N} \sum_{t=0}^{T} \nabla_\theta \log \pi_\theta (a_{it} | s_{it}) \sum_{t'=t}^T \gamma^{t'-t}r_i(t')
\end{equation}

You will be implementing final expression in this assignment!



The process of REINFOCE algorithm follows:

1. Collect samples from current policy $\pi_\theta(s)$ by executing rollouts of the environment.
2. Calculate an estimate for the expected return at state $s_t$. 
3. Compute the log-likelihood of each action that was performed by the policy at every given step.
4. Estimate the gradient and update the parameters of policy using gradient-based technique.
5. Repeat steps 1-4 for a number of training iterations.

***Your task*** is to fill out the skeleton code for REINFORCE algorithm,

1. Complete the 'log_likelihoods' method to compute gradient of policy, $\nabla_{\theta}\pi_{\theta}$ for diagonal Guassian policy. 

2. Complete the 'compute_expected_return' method to calculate the reward-to-go, $\sum_{t^{\prime}=t}^{T}$. 



In [11]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time

class REINFORCE(PolicyOpt):
    def train(self, num_iterations=1000, steps_per_iteration=1000, learning_rate=3e-5, gamma=0.95, **kwargs):
        """
        Vanilla REINFORCE (Monte-Carlo policy gradient) with a Gaussian policy on continuous actions.
        No baseline, no entropy bonus, no normalization.
        """
        self.gamma = gamma
        self.optimizer = optim.Adam(list(self.policy_net.parameters()) + [self.log_std], lr=float(learning_rate))

        episode_returns = []
        env = self.env

        total_steps = 0
        for it in range(num_iterations):
            
            # clamp exploration scale each iteration to avoid σ->0 or exploding σ
            with torch.no_grad():
                self.log_std.data.clamp_(-2.0, 1.0)

            # Collect trajectories until reaching steps_per_iteration
            batch_states = []
            batch_actions = []
            batch_returns = []
            steps_in_batch = 0

            while steps_in_batch < steps_per_iteration:
                # start new episode
                out = env.reset()
                obs = out[0] if isinstance(out, tuple) else out
                ep_states, ep_actions, ep_rewards = [], [], []
                done = False
                while not done:
                    s_t = torch.as_tensor(obs, dtype=torch.float32).unsqueeze(0)
                    dist = self._dist(s_t)
                    a_t = dist.sample() if self.stochastic else dist.mean
                    a_np = a_t.squeeze(0).detach().cpu().numpy()
                    a_np = np.clip(a_np, self.act_low.numpy(), self.act_high.numpy())
                    step_out = env.step(a_np)
                    if len(step_out) == 5:
                        obs_next, r, terminated, truncated, info = step_out
                        done = bool(terminated or truncated)
                    else:
                        obs_next, r, done, info = step_out
                        done = bool(done)

                    ep_states.append(obs)
                    ep_actions.append(a_np)
                    ep_rewards.append(float(r))

                    obs = obs_next
                    steps_in_batch += 1
                    if done:
                        # compute reward-to-go for this episode and log return
                        G = np.zeros_like(ep_rewards, dtype=np.float32)
                        g = 0.0
                        for t in range(len(ep_rewards)-1, -1, -1):
                            g = ep_rewards[t] + gamma * g
                            G[t] = g
                        batch_states.extend(ep_states)
                        batch_actions.extend(ep_actions)
                        batch_returns.extend(G.tolist())
                        episode_returns.append(sum(ep_rewards))
                        break

            # Convert to tensors
            s_b = torch.as_tensor(np.array(batch_states, dtype=np.float32))
            a_b = torch.as_tensor(np.array(batch_actions, dtype=np.float32))
            G_b = torch.as_tensor(np.array(batch_returns, dtype=np.float32))

            # Policy loss = -E[ log pi(a|s) * G ]
            dist_b = self._dist(s_b)
            logp = dist_b.log_prob(a_b).sum(dim=-1)
            loss = -(logp * G_b).mean()

            self.optimizer.zero_grad(set_to_none=True)
            loss.backward()
            # clip gradients to avoid rare spikes
            torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
            self.optimizer.step()

            total_steps += steps_in_batch
            if (it+1) % max(1, num_iterations//10) == 0:
                print(f"[Iter {it+1}/{num_iterations}] Steps: {total_steps} | Loss: {float(loss.item()):.4f} | EpRet(last): {episode_returns[-1]:.1f}")

        return episode_returns


Check your 'log_likelihoods' method by running below cell:

In [12]:
alg = REINFORCE(TEST_ENV, stochastic=True)
pass  # PyTorch version: call log_likelihoods_eval directly
# collect a sample output for a given input state
input_s = [[0, 0, 0], [0, 1, 2], [1, 2, 3]]
input_a = [[0], [1], [2]]

# Check
computed = alg.log_likelihoods_eval(states=input_s, actions=input_a)

print('log_likelihoods:', computed)


log_likelihoods: [ -0.4189842  -3.232242  -10.284169 ]


Test your 'compute_expected_return' by running below cell:

In [13]:
# 1. Test the non-normalized case
alg = REINFORCE(TEST_ENV, stochastic=True)
alg.gamma = 1.0
    
input_1 = [{"reward": [1, 1, 1, 1]},
           {"reward": [1, 1, 1, 1]}]
vs_1 = alg.compute_expected_return(samples=input_1)
ans_1 = np.array([4, 3, 2, 1, 4, 3, 2, 1])

if np.linalg.norm(vs_1 - ans_1) < 1e-3:
    print('Great job!')
else:
    print('Check your implementation (compute_expected_return)')

    

Great job!


## 1.4 Testing your algorithm

When you are ready, test your policy gradient algorithms on the *Pendulum-v0* environment in the cell below. *Pendulum-v0* environment is similar to *off-shore wind power*, the goal here is to maintain the Pendulum is upright using control input. The best policy should get around -200 scores. ***Your task*** is to run your REINFORCE algorithm and plot the result!


In [15]:
# set this number as 1 for testing your algorithm, and 3 for plotting
NUM_TRIALS = 3

# ===========================================================================
# Do not modify below line
# ===========================================================================

# we will test the algorithms on the Pendulum-v1 gym environment

env = gym.make("Pendulum-v0")

# train on the REINFORCE algorithm
import numpy as np
r = []
for i in range(NUM_TRIALS):
    print("\n==== Training Run {} ====".format(i))
    alg = REINFORCE(env, stochastic=True)
    res = alg.train(learning_rate=0.005, gamma=0.95, num_iterations=500, steps_per_iteration=5000)
    r.append(np.array(res))
    alg = None

# save results
np.savetxt("InvertedPendulum_results.csv", np.array(r), delimiter=",")


==== Training Run 0 ====
[Iter 50/500] Steps: 250000 | Loss: -532.3245 | EpRet(last): -1471.3
[Iter 100/500] Steps: 500000 | Loss: -26187.5176 | EpRet(last): -1497.6
[Iter 150/500] Steps: 750000 | Loss: -234914.4375 | EpRet(last): -1214.3
[Iter 200/500] Steps: 1000000 | Loss: -2514733.2500 | EpRet(last): -1215.1
[Iter 250/500] Steps: 1250000 | Loss: -4825278.0000 | EpRet(last): -1656.1
[Iter 300/500] Steps: 1500000 | Loss: -7234583.0000 | EpRet(last): -1201.2
[Iter 350/500] Steps: 1750000 | Loss: -11491135.0000 | EpRet(last): -1491.4
[Iter 400/500] Steps: 2000000 | Loss: -14423032.0000 | EpRet(last): -1169.5
[Iter 450/500] Steps: 2250000 | Loss: -18495330.0000 | EpRet(last): -1319.2
[Iter 500/500] Steps: 2500000 | Loss: -24073502.0000 | EpRet(last): -1346.3

==== Training Run 1 ====
[Iter 50/500] Steps: 250000 | Loss: -111.8563 | EpRet(last): -1127.3
[Iter 100/500] Steps: 500000 | Loss: -130.7528 | EpRet(last): -1178.9
[Iter 150/500] Steps: 750000 | Loss: -3266.5950 | EpRet(last): -15

In [16]:
# collect saved results
import numpy as np
r1 = np.genfromtxt("InvertedPendulum_results.csv", delimiter=",")
all_results = [r1]
labels = ["REINFORCE"]

##############################################################
# Plot your Policy Gradient results below
##############################################################
?


IPython -- An enhanced Interactive Python

IPython offers a fully compatible replacement for the standard Python
interpreter, with convenient shell features, special commands, command
history mechanism and output results caching.

At your system command line, type 'ipython -h' to see the command line
options available. This document only describes interactive features.

GETTING HELP
------------

Within IPython you have various way to access help:

  ?         -> Introduction and overview of IPython's features (this screen).
  object?   -> Details about 'object'.
  object??  -> More detailed, verbose information about 'object'.
  %quickref -> Quick reference of all IPython specific syntax and magics.
  help      -> Access Python's own help system.

If you are in terminal IPython you can quit this screen by pressing `q`.


MAIN FEATURES
-------------

* Access to the standard Python help with object docstrings and the Python
  manuals. Simply type 'help' (no quotes) to invoke it.

* Ma

In [21]:

# --- PolicyOpt (PyTorch) + REINFORCE (vanilla), CUDA-ready ---
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal

# Device
try:
    DEVICE
except NameError:
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

class PolicyOpt(object):
    def __init__(self, env, linear=False, stochastic=True, hidden_size=32, nonlinearity=None):
        """Gaussian policy π(a|s): mean = MLP(s), diag std = exp(log_std)."""
        self.env = env
        self.stochastic = stochastic
        self.obs_dim = int(np.prod(env.observation_space.shape))
        self.act_dim = int(np.prod(env.action_space.shape))

        # Action bounds (on device) + cached NumPy copies
        self.act_low  = torch.as_tensor(env.action_space.low,  dtype=torch.float32, device=DEVICE)
        self.act_high = torch.as_tensor(env.action_space.high, dtype=torch.float32, device=DEVICE)
        self.act_low_np  = self.act_low.detach().cpu().numpy()
        self.act_high_np = self.act_high.detach().cpu().numpy()

        # Policy network
        layers, last = [], self.obs_dim
        if linear:
            layers.append(nn.Linear(last, self.act_dim))
        else:
            layers += [nn.Linear(last, hidden_size), nn.Tanh(), nn.Linear(hidden_size, self.act_dim)]
        self.policy_net = nn.Sequential(*layers).to(DEVICE)

        # State-independent log std (trainable)
        self.log_std = nn.Parameter(torch.full((self.act_dim,), -0.5, device=DEVICE))

        self.optimizer = None
        self.gamma = 0.99

    def _dist(self, obs_tensor: torch.Tensor) -> Normal:
        mu = self.policy_net(obs_tensor)
        std = torch.exp(self.log_std).expand_as(mu)
        return Normal(mu, std)

    def compute_action(self, s):
        """Return an action for a single state or batch of states."""
        s_t = torch.as_tensor(np.array(s, dtype=np.float32), device=DEVICE)
        if s_t.ndim == 1:
            s_t = s_t.unsqueeze(0)
        with torch.no_grad():
            dist = self._dist(s_t)
            a = dist.sample() if self.stochastic else dist.mean
        a = a.clamp(self.act_low, self.act_high)
        return a.detach().cpu().numpy()

    def log_likelihoods_eval(self, states, actions):
        """Return log π(a|s) for each (s,a) pair: shape [T]."""
        s_t = torch.as_tensor(np.array(states, dtype=np.float32), device=DEVICE)
        a_t = torch.as_tensor(np.array(actions, dtype=np.float32), device=DEVICE)
        if s_t.ndim == 1: s_t = s_t.unsqueeze(0)
        if a_t.ndim == 1: a_t = a_t.unsqueeze(0)
        dist = self._dist(s_t)
        logp = dist.log_prob(a_t).sum(dim=-1)
        return logp.detach().cpu().numpy()

    def compute_expected_return(self, samples, normalize=False):
        """Concatenate reward-to-go across all episodes."""
        all_G = []
        for ep in samples:
            r = np.asarray(ep["reward"], dtype=np.float32)
            G = np.zeros_like(r, dtype=np.float32)
            g = 0.0
            for t in range(len(r) - 1, -1, -1):
                g = r[t] + float(self.gamma) * g
                G[t] = g
            all_G.append(G)
        out = np.concatenate(all_G, axis=0)
        if normalize and out.size > 1:
            out = (out - out.mean()) / (out.std() + 1e-8)
        return out


class REINFORCE(PolicyOpt):
    def train(
        self,
        num_iterations: int = 1000,
        steps_per_iteration: int = 1000,
        learning_rate: float = 3e-5,   # keep float; int(1e-4) == 0!
        gamma: float = 0.95,
        **kwargs,
    ):
        """Vanilla REINFORCE (no baseline/entropy/normalization)."""
        import time
        self.gamma = gamma
        self.optimizer = optim.Adam(
            list(self.policy_net.parameters()) + [self.log_std],
            lr=float(learning_rate),
        )

        episode_returns = []
        env = self.env
        total_steps = 0

        for it in range(num_iterations):
            
            # clamp exploration scale each iteration to avoid σ->0 or exploding σ
            with torch.no_grad():
                self.log_std.data.clamp_(-2.0, 1.0)

            t0 = time.perf_counter()
            batch_states, batch_actions, batch_returns = [], [], []
            steps_in_batch = 0

            while steps_in_batch < steps_per_iteration:
                out = env.reset()
                obs = out[0] if isinstance(out, tuple) else out

                ep_states, ep_actions, ep_rewards = [], [], []
                done = False

                low_np, high_np = self.act_low_np, self.act_high_np

                while not done:
                    s_t = torch.from_numpy(np.asarray(obs, np.float32)).to(DEVICE).unsqueeze(0)
                    with torch.no_grad():
                        dist = self._dist(s_t)
                        a_t = dist.sample() if self.stochastic else dist.mean

                    a_np = a_t.squeeze(0).detach().cpu().numpy()
                    a_np = np.clip(a_np, low_np, high_np)

                    step_out = env.step(a_np)
                    if len(step_out) == 5:
                        obs_next, r, terminated, truncated, info = step_out
                        done = bool(terminated or truncated)
                    else:
                        obs_next, r, done, info = step_out
                        done = bool(done)

                    ep_states.append(obs)
                    ep_actions.append(a_np)
                    ep_rewards.append(float(r))

                    obs = obs_next
                    steps_in_batch += 1

                    if done:
                        G = np.zeros_like(ep_rewards, dtype=np.float32)
                        g = 0.0
                        for t in range(len(ep_rewards) - 1, -1, -1):
                            g = ep_rewards[t] + gamma * g
                            G[t] = g

                        batch_states.extend(ep_states)
                        batch_actions.extend(ep_actions)
                        batch_returns.extend(G.tolist())
                        episode_returns.append(sum(ep_rewards))
                        break

            # Gradient step on the batch
            s_b = torch.as_tensor(np.array(batch_states,  dtype=np.float32), device=DEVICE)
            a_b = torch.as_tensor(np.array(batch_actions, dtype=np.float32), device=DEVICE)
            G_b = torch.as_tensor(np.array(batch_returns, dtype=np.float32), device=DEVICE)

            dist_b = self._dist(s_b)
            logp   = dist_b.log_prob(a_b).sum(dim=-1)
            loss   = -(logp * G_b).mean()

            self.optimizer.zero_grad(set_to_none=True)
            loss.backward()
            # clip gradients to avoid rare spikes
            torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
            self.optimizer.step()

            total_steps += steps_in_batch
            dt = time.perf_counter() - t0
            if (it + 1) % max(1, num_iterations // 10) == 0:
                print(f"[Iter {it+1}/{num_iterations}] "
                      f"Steps: {total_steps} (+{steps_in_batch} @ {steps_in_batch/max(dt,1e-6):.0f} steps/s) | "
                      f"Loss: {float(loss.item()):.4f} | EpRet(last): {episode_returns[-1]:.1f}")
        return episode_returns


Using device: cuda
